# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [343]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [344]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import numpy as np

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [345]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [346]:
#Produto escolhido:
produto = 'tang'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [347]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [348]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [367]:

tabela_tang = pd.read_excel('tang.xlsx', sheetname = 'Treinamento') # lendo a planilha
tabela_tang.categoria =tabela_tang.categoria.astype('category')

#limpando a tabela:

tabela_tang.Treinamento = tabela_tang.Treinamento.replace('\.','',regex = True)
tabela_tang.Treinamento = tabela_tang.Treinamento.replace(',','',regex = True)
tabela_tang.Treinamento = tabela_tang.Treinamento.replace('@','',regex = True)
tabela_tang.Treinamento = tabela_tang.Treinamento.replace('/','',regex = True)
tabela_tang.Treinamento = tabela_tang.Treinamento.replace(':','',regex = True)
tabela_tang.Treinamento = tabela_tang.Treinamento.replace(';','',regex = True)
tabela_tang.Treinamento = tabela_tang.Treinamento.replace('\(','',regex = True)
tabela_tang.Treinamento = tabela_tang.Treinamento.replace('\)','',regex = True)
tabela_tang.Treinamento = tabela_tang.Treinamento.replace('!','',regex = True)
tabela_tang.Treinamento = tabela_tang.Treinamento.replace('_','',regex = True)
tabela_tang.Treinamento = tabela_tang.Treinamento.replace('-','',regex = True)
tabela_tang.Treinamento = tabela_tang.Treinamento.replace('\n','',regex = True)
tabela_tang.Treinamento = tabela_tang.Treinamento.replace(']','',regex = True)
#tabela_tang.Treinamento = tabela_tang.Treinamento.replace('[','',regex = True)


def criacao(lista,tabela):
    
    for i,linha in tabela.iterrows():
        palavras = linha[0].split(" ")
        for i in palavras:
            lista.append(i)
    serie = pd.Series(lista, name = 'Palavras')
    return serie

# separando relevantes:

#total_de_palavras = len(lista_de_palavras)

lista_de_palavras =[]
total = criacao(lista_de_palavras, tabela_tang)
T_V = total.value_counts()


Relevantes = tabela_tang[tabela_tang.categoria == "relevante"].iloc[:,:]
lista_de_relevantes = []
R = criacao(lista_de_relevantes,Relevantes)
R_V = R.value_counts()

Irrelevantes = tabela_tang[tabela_tang.categoria == "irrelevante"].iloc[:,:]
lista_de_irrelevantes = []
I = criacao(lista_de_irrelevantes,Irrelevantes)
I_V = I.value_counts()

total_p_r = len(R) + len(R_V)
total_p_ir = len(I) + len(I_V)

P_tg= tabela_tang.categoria.value_counts(True)

lista_P_r = []
lista_P_ir = []

for i,linha in tabela_tang.iterrows():
    P_multi_r = 1
    P_multi_ir = 1
    palavras = linha[0].split(" ")
    
   # for i in range(len(palavras)):
      #  palavras[a]
        
    for i in palavras:
        P_r = (T_V[i] + 1) /total_p_r
        P_palavra_r = P_r * P_tg["relevante"]
        
        P_ir = (T_V[i] + 1) /total_p_ir
        P_palavra_ir = P_ir * P_tg["irrelevante"]
        
        P_multi_r *= P_palavra_r
        P_multi_ir *= P_palavra_ir
        
    lista_P_r.append(P_multi_r)
    lista_P_ir.append(P_multi_ir)

lista_frase = []

for i in range(len(lista_P_r)):
    a = lista_P_r[i]
    b = lista_P_ir[i]
    
    if a > b: 
        lista_frase.append('relevante')

    elif b > a: 

        lista_frase.append('irrelevante')
        
        
serie_r = pd.Series(lista_frase, name = 'RouIr')

#tabela_tang.loc[serie_r]

tabela_tang["RouIR"] = lista_frase

tabela_tang.RouIR =tabela_tang.RouIR.astype('category')


tabela_tang.loc[(tabela_tang.categoria == "relevante") & (tabela_tang.RouIR == "relevante"), "Verificando"] = "P-V"
tabela_tang.loc[(tabela_tang.categoria == "irrelevante") & (tabela_tang.RouIR == "relevante"), "Verificando"] = "P-F"
tabela_tang.loc[(tabela_tang.categoria == "irrelevante") & (tabela_tang.RouIR == "irrelevante"), "Verificando"] = "N-V"
tabela_tang.loc[(tabela_tang.categoria == "relevante") & (tabela_tang.RouIR == "irrelevante"), "Verificando"] = "N-F"



tabela_tang.head()

1942
3675


,Treinamento,categoria,RouIR,Verificando
0,wu tang jeru the damaja montel jordan cypre...,irrelevante,relevante,P-F
1,minha filha né gente criei com muito tang http...,relevante,relevante,P-V
2,rt ingrdgabrielle falo q dessa água não bebere...,relevante,relevante,P-V
3,winxebloom nojo mesmo que isso muito tang eca,relevante,relevante,P-V
4,pessoas que vão no mercado e gastam 18 pila em...,relevante,relevante,P-V


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [363]:

tabela_tang = pd.read_excel('tang.xlsx',sheetname = 'Teste') # lendo a planilha
tabela_tang.categoria = tabela_tang.categoria.astype('category')

#limpando a tabela:

tabela_tang.Teste = tabela_tang.Teste.replace('\.','',regex = True)
tabela_tang.Teste = tabela_tang.Teste.replace(',','',regex = True)
tabela_tang.Teste = tabela_tang.Teste.replace('@','',regex = True)
tabela_tang.Teste = tabela_tang.Teste.replace('/','',regex = True)
tabela_tang.Teste = tabela_tang.Teste.replace(':','',regex = True)
tabela_tang.Teste = tabela_tang.Teste.replace(';','',regex = True)
tabela_tang.Teste = tabela_tang.Teste.replace('\(','',regex = True)
tabela_tang.Teste = tabela_tang.Teste.replace('\)','',regex = True)
tabela_tang.Teste = tabela_tang.Teste.replace('!','',regex = True)
tabela_tang.Teste = tabela_tang.Teste.replace('_','',regex = True)
tabela_tang.Teste = tabela_tang.Teste.replace('-','',regex = True)
tabela_tang.Teste = tabela_tang.Teste.replace('\n','',regex = True)
tabela_tang.Teste = tabela_tang.Teste.replace(']','',regex = True)
#tabela_tang.Treinamento = tabela_tang.Treinamento.replace('[','',regex = True)


def criacao(lista,tabela):
    
    for i,linha in tabela.iterrows():
        palavras = linha[0].split(" ")
        for i in palavras:
            lista.append(i)
    serie = pd.Series(lista, name = 'Palavras')
    return serie

# separando relevantes:

lista_de_palavras =[]
total = criacao(lista_de_palavras, tabela_tang)
T_V = total.value_counts()


Relevantes = tabela_tang[tabela_tang.categoria == "relevante"].iloc[:,:]
lista_de_relevantes = []
R = criacao(lista_de_relevantes,Relevantes)
R_V = R.value_counts()

Irrelevantes = tabela_tang[tabela_tang.categoria == "irrelevante"].iloc[:,:]
lista_de_irrelevantes = []
I = criacao(lista_de_irrelevantes,Irrelevantes)
I_V = I.value_counts()

total_p_r = len(R) + len(R_V)
total_p_ir = len(I) + len(I_V)


P_tg= tabela_tang.categoria.value_counts(True)

lista_P_r = []
lista_P_ir = []

for i,linha in tabela_tang.iterrows():
    P_multi_r = 1
    P_multi_ir = 1
    palavras = linha[0].split(" ")
    for i in palavras:
        P_r = (T_V[i] + 1) /total_p_r
        P_palavra_r = P_r * P_tg["relevante"]
        
        P_ir = (T_V[i] + 1) /total_p_ir
        P_palavra_ir = P_ir * P_tg["irrelevante"]
        
        P_multi_r *= P_palavra_r
        P_multi_ir *= P_palavra_ir
        
    lista_P_r.append(P_multi_r)
    lista_P_ir.append(P_multi_ir)

lista_frase = []

for i in range(len(lista_P_r)):
    a = lista_P_r[i]
    b = lista_P_ir[i]
    
    if a > b: 
        lista_frase.append('relevante')

    elif b > a: 

        lista_frase.append('irrelevante')
        
        
serie_r = pd.Series(lista_frase, name = 'RouIr')

#tabela_tang.loc[serie_r]

tabela_tang["RouIR"] = lista_frase

tabela_tang.RouIR =tabela_tang.RouIR.astype('category')


tabela_tang.loc[(tabela_tang.categoria == "relevante") & (tabela_tang.RouIR == "relevante"), "Verificando"] = "P-V"
tabela_tang.loc[(tabela_tang.categoria == "irrelevante") & (tabela_tang.RouIR == "relevante"), "Verificando"] = "P-F"
tabela_tang.loc[(tabela_tang.categoria == "irrelevante") & (tabela_tang.RouIR == "irrelevante"), "Verificando"] = "N-V"
tabela_tang.loc[(tabela_tang.categoria == "relevante") & (tabela_tang.RouIR == "irrelevante"), "Verificando"] = "N-F"



tabela_tang.head()

,Teste,categoria,RouIR,Verificando
0,esse show do wu tang é foda demais n me canso ...,irrelevante,relevante,P-F
1,sdds de quando eu tomava raiska com tang,irrelevante,relevante,P-F
2,myfavslox jackgilinsky cheirou po tang so pode,irrelevante,relevante,P-F
3,finallygui vai derreter meu coração por causa ...,irrelevante,relevante,P-F
4,o macho do meu tio ta mandando minha vó fazer ...,irrelevante,relevante,P-F


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
